In [ ]:
#Algo asi hay que hacer en la tarea
import cmath
import numpy as np
#Primero que nada necesitamos encontrar el valor de las matrices locales de las lineas, por lo que aquí se colocan los datos de la linea y despues se modelan por el modelo Pi para luego poder sumarlas al Y_buss
Ybuss = np.zeros((6, 6), dtype=complex)
Lista_datos_Lineas= [("1-2",0.031,0.126,0.062),
                     ("1-5",0.031,0.126,0.062),
                     ("2-3",0.063,0.252,0.122),
                     ("3-5",0.053,0.210,0.102),
                     ("3-6",0.084,0.236,0.164)]; # (linea,r,x,b)
for i in Lista_datos_Lineas:
    L=150
    Linea=i[0].split("-")
    Linea_a=int(Linea[0])-1
    Linea_b=int(Linea[1])-1
    z = complex(i[1],i[2])
    y = complex(0,i[3])
    Zc = cmath.sqrt(z/y)
    gama = cmath.sqrt(z*y)
    Z_modelo=Zc*gama*L
    Y_Serie=1/Z_modelo

    Y_Shunt=gama*L/(Zc*2)
    Y_Shunt=Y_Shunt.imag*1j
    
    A=Y_Serie+Y_Shunt
    B=-Y_Serie
    C=B
    D=A

    coordenadas = [(Linea_a, Linea_a), (Linea_a, Linea_b), 
                   (Linea_b, Linea_a), (Linea_b, Linea_b)]
    valores_a_sumar = [A, B, C, D]
    for coord, valor in zip(coordenadas, valores_a_sumar):
        fila, columna = coord
        Ybuss[fila, columna] += valor

############################Trafos######################################

Lista_datos_Trafos= [("4-5",0,0.1,0),
                     ("4-6",0,0.1,0)]; # (linea,r,x,b)
for i in Lista_datos_Trafos:
    Linea=i[0].split("-")
    Linea_a=int(Linea[0])-1
    Linea_b=int(Linea[1])-1

    Z_trafo=complex(i[1],i[2])
    Y_trafo=1/Z_trafo
    A=Y_trafo
    B=-A
    C=B
    D=A

    valores_a_sumar = [A, B, C, D]
    coordenadas = [(Linea_a, Linea_a), (Linea_a, Linea_b), (Linea_b, Linea_a), (Linea_b, Linea_b)]
    for coord, valor in zip(coordenadas, valores_a_sumar):
        fila, columna = coord
        Ybuss[fila, columna] += valor

############################Generación######################################

Lista_datos_Trafos= [("1",0,1.1,0),
                     ("3",0,1.3,0)]; # (linea,r,x,b)
for i in Lista_datos_Trafos:
    Linea=i[0].split("-")
    Linea_a=int(Linea[0])-1
    Z_trafo=complex(i[1],i[2])
    Y_trafo=1/Z_trafo

    A=Y_trafo
    coordenadas = [(Linea_a, Linea_a)]
    valores_a_sumar = [A]
    for coord, valor in zip(coordenadas, valores_a_sumar):
        fila, columna = coord
        Ybuss[fila, columna] += valor
print(Ybuss)
#Prueba

In [24]:
# ------------- SISTEMA - CERTAMEN 2 ------------- 
import pandapower as pp
net = pp.create_empty_network()

#Parametros de Línea
r = 0.099 #(Ohm/km)
x = 0.156 #(Ohm/km)
c = 125   #(nF/km) 
p = 1/4   #(W)

#Barras
b1 = pp.create_bus(net, vn_kv=0.015, name="Barra 1")
b2 = pp.create_bus(net, vn_kv=0.22, name="Barra 2")
b3 = pp.create_bus(net, vn_kv=0.22, name="Barra 3")
b4 = pp.create_bus(net, vn_kv=0.22, name="Barra 4")
b5 = pp.create_bus(net, vn_kv=0.22, name="Barra 5")
b6 = pp.create_bus(net, vn_kv=0.22, name="Barra 6")
b7 = pp.create_bus(net, vn_kv=0.22, name="Barra 7")
b8 = pp.create_bus(net, vn_kv=0.22, name="Barra 8")

#Elementos De Barra
pp.create_ext_grid(net, bus=b1, vm_pu=1.00, name="Fuente Externa") #FUENTE
pp.create_load(net, bus=b3, p_mw=(p*0.20)/1E6, q_mvar=0, name="Carga 3")     #CARGAS
pp.create_load(net, bus=b4, p_mw=(p*0.35)/1E6, q_mvar=0, name="Carga 4")
pp.create_load(net, bus=b5, p_mw=(p*0.15)/1E6, q_mvar=0, name="Carga 5")
pp.create_load(net, bus=b6, p_mw=(p*0.12)/1E6, q_mvar=0, name="Carga 6")
pp.create_load(net, bus=b7, p_mw=(p*0.48)/1E6, q_mvar=0, name="Carga 7")
pp.create_load(net, bus=b8, p_mw=(p*0.10)/1E6, q_mvar=0, name="Carga 8")

#Elementos De Rama
tid = pp.create_transformer_from_parameters(net, hv_bus=b2, lv_bus=b1, name="Trafo", sn_mva=0.1,
                                            vn_hv_kv=0.22, vn_lv_kv=0.015, vk_percent=4,vkr_percent=1.0794, 
                                            pfe_kw=1.18, i0_percent=0.1873, shift_grado=0)
lin_params = {
    'r_ohm_per_km': 0.099,
    'x_ohm_per_km': 0.156,
    'c_nf_per_km': 125,
    'max_i_ka': 0.1
}
#L2-3
pp.create_line_from_parameters(net, name='Linea 2-3(A)', length_km = 9, from_bus = b2, to_bus = b3, **lin_params)
pp.create_line_from_parameters(net, name='Linea 2-3(B)', length_km = 9, from_bus = b2, to_bus = b3, **lin_params)
#L3-4
pp.create_line_from_parameters(net, name='Linea 3-4(A)', length_km = 15, from_bus = b3, to_bus = b4, **lin_params) 
pp.create_line_from_parameters(net, name='Linea 3-4(B)', length_km = 15, from_bus = b3, to_bus = b4, **lin_params)
#L4-5
pp.create_line_from_parameters(net, name='Linea 4-5(A)', length_km = 20, from_bus = b4, to_bus = b5, **lin_params)
pp.create_line_from_parameters(net, name='Linea 4-5(B)', length_km = 20, from_bus = b4, to_bus = b5, **lin_params)
#L5-6
pp.create_line_from_parameters(net, name='Linea 5-6(A)', length_km = 10, from_bus = b5, to_bus = b6, **lin_params)
pp.create_line_from_parameters(net, name='Linea 5-6(B)', length_km = 10, from_bus = b5, to_bus = b6, **lin_params)
#L6-7
pp.create_line_from_parameters(net, name='Linea 6-7(A)', length_km = 5, from_bus = b6, to_bus = b7, **lin_params)
pp.create_line_from_parameters(net, name='Linea 6-7(B)', length_km = 5, from_bus = b6, to_bus = b7, **lin_params)
#L7-8
pp.create_line_from_parameters(net, name='Linea 7-8(A)', length_km = 30, from_bus = b7, to_bus = b8, **lin_params)
pp.create_line_from_parameters(net, name='Linea 7-8(B)', length_km = 30, from_bus = b7, to_bus = b8, **lin_params)
#L8-2
pp.create_line_from_parameters(net, name='Linea 8-2(A)', length_km = 10, from_bus = b8, to_bus = b2, **lin_params) 
pp.create_line_from_parameters(net, name='Linea 8-2(B)', length_km = 10, from_bus = b8, to_bus = b2, **lin_params)

#Flujo De Energía
pp.runpp(net)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)


In [29]:
# Crear mapeo de índices de barras a nombres
bus_indices = {b1: "Barra 1", b2: "Barra 2", b3: "Barra 3", b4: "Barra 4", b5: "Barra 5", b6: "Barra 6", b7: "Barra 7", b8: "Barra 8"}

# Imprimir resultados de las barras con nombres
print("Resultados de las barras:")
for index, row in net.res_bus.iterrows():
    print(f"{bus_indices.get(index, 'Unknown')}: {row}")

# Imprimir resultados de las líneas
#print("\nResultados de las líneas:")
#print(net.res_line)

# Imprimir resultados del transformador
#print("\nResultados del transformador:")
#print(net.res_trafo)

Resultados de las barras:
Barra 1: vm_pu        1.000000
va_degree    0.000000
p_mw        -0.001182
q_mvar       0.000378
Name: 0, dtype: float64
Barra 2: vm_pu        1.000082
va_degree   -0.015392
p_mw         0.000000
q_mvar       0.000000
Name: 1, dtype: float64
Barra 3: vm_pu        1.002571e+00
va_degree   -1.065285e-01
p_mw         5.000000e-08
q_mvar       0.000000e+00
Name: 2, dtype: float64
Barra 4: vm_pu        1.005618e+00
va_degree   -2.175435e-01
p_mw         8.750000e-08
q_mvar       0.000000e+00
Name: 3, dtype: float64
Barra 5: vm_pu        1.007529e+00
va_degree   -2.868610e-01
p_mw         3.750000e-08
q_mvar       0.000000e+00
Name: 4, dtype: float64
Barra 6: vm_pu        1.007560e+00
va_degree   -2.880072e-01
p_mw         3.000000e-08
q_mvar       0.000000e+00
Name: 5, dtype: float64
Barra 7: vm_pu        1.007344e+00
va_degree   -2.802119e-01
p_mw         1.200000e-07
q_mvar       0.000000e+00
Name: 6, dtype: float64
Barra 8: vm_pu        1.002816e+00
va_degree   